In [1]:
import requests

https://api.weirdgloop.org/exchange/history/osrs/latest?id=4151

In [2]:
# [(Leaf, Seed)]
herbs = [
    ("Toadflax", "Toadflax seed"), ("Harralander", "Harralander seed"), ("Tarromin", "Tarromin seed"),
    ("Marrentill", "Marrentill seed"), ("Ranarr weed", "Ranarr seed"),
    ("Irit leaf", "Irit seed"), ("Avantoe", "Avantoe seed"), ("Kwuarm", "Kwuarm seed")
]

In [3]:
latest_url = "https://api.weirdgloop.org/exchange/history/osrs/latest"
# {Leaf: (Leaf price, Seed price)}
results = {}

for leaf, seed in herbs:
    leaf_data, seed_data = None, None
    # Leaf
    leaf_response = requests.get(latest_url, params={"name":leaf})
    if leaf_response.status_code == 200:
        leaf_data = leaf_response.json()
        print(leaf_data)
    # Seed
    seed_response = requests.get(latest_url, params={"name":seed})
    if seed_response.status_code == 200:
        seed_data = seed_response.json()
    if not leaf_data or not seed_data:
        print(f"Error with {leaf}:\n\t{leaf_response.status_code}\n\t{seed_response.status_code}")
    else:
        results[leaf] = (leaf_data[leaf]["price"], seed_data[seed]["price"])

{'Toadflax': {'id': '2998', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 3767, 'volume': 265922}}
{'Harralander': {'id': '255', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 728, 'volume': 346790}}
{'Tarromin': {'id': '253', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 409, 'volume': 248666}}
{'Marrentill': {'id': '251', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 597, 'volume': 200716}}
{'Ranarr weed': {'id': '257', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 6984, 'volume': 249141}}
{'Irit leaf': {'id': '259', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 2819, 'volume': 92029}}
{'Avantoe': {'id': '261', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 3746, 'volume': 124651}}
{'Kwuarm': {'id': '263', 'timestamp': '2025-07-10T06:47:23.000Z', 'price': 3813, 'volume': 147928}}


"The average yield of a standard patch (Falador, Ardougne, Canifis) was 8.56 herbs"
https://oldschoolrunescape.fandom.com/wiki/Herb

In [4]:
profits = {key: round((value[0]*8.56)-value[1]) for key, value in results.items()}
sorted(profits.items(), key=lambda item: item[1], reverse=True)

[('Avantoe', 31320),
 ('Toadflax', 31288),
 ('Ranarr weed', 30947),
 ('Kwuarm', 30246),
 ('Irit leaf', 24054),
 ('Harralander', 6198),
 ('Marrentill', 5090),
 ('Tarromin', 3489)]

It does look like the "latest" api endpoint may only be pulling once a day - for the last three hours it's had the same timestamp. Possibly look at other APIs out there? Additionally, need to decide how to determine which is the best herb to grow using more than just the singular latest price point - it should take into account at least the past few hours if not the past day's prices.

I wonder if there's room to use CUSUM here?